In [0]:
from pyspark.sql import functions as F
import datetime as dt

# Source tables
docs = spark.table("knowledgehub_lakehouse.silver.docs_clean")
events = spark.table("knowledgehub_lakehouse.silver.access_events_clean")

# Collect all dates dynamically
all_dates = (
    docs.select(F.to_date("created_ts").alias("d"))
    .union(docs.select(F.to_date("updated_ts").alias("d")))
    .union(events.select(F.to_date("event_ts").alias("d")))
    .where("d is not null")
)

min_date = all_dates.agg(F.min("d")).first()[0]
max_date = all_dates.agg(F.max("d")).first()[0]

# Buffer for analytics
start_date = (min_date - dt.timedelta(days=30)).isoformat()
end_date   = (max_date + dt.timedelta(days=365)).isoformat()

# -------------------------------
# DIM DATE
# -------------------------------
dim_date = (
    spark.sql(f"""
        SELECT explode(
            sequence(
                to_date('{start_date}'),
                to_date('{end_date}'),
                interval 1 day
            )
        ) AS date
    """)
    # Surrogate Key
    .withColumn("date_key", F.date_format("date", "yyyyMMdd").cast("int"))

    # Day attributes
    .withColumn("day", F.dayofmonth("date"))
    .withColumn("day_name", F.date_format("date", "EEEE"))
    .withColumn("day_of_week", F.dayofweek("date"))

    # Week attributes
    .withColumn("week_of_year", F.weekofyear("date"))

    # Month attributes
    .withColumn("month", F.month("date"))
    .withColumn("month_name", F.date_format("date", "MMMM"))

    # Quarter attributes
    .withColumn("quarter", F.quarter("date"))

    # Year attributes
    .withColumn("year", F.year("date"))

    # Flags
    .withColumn(
        "is_weekend",
        F.when(F.dayofweek("date").isin([1, 7]), 1).otherwise(0)
    )
)

# Write to Gold
dim_date.write.mode("overwrite").format("delta").saveAsTable(
    "knowledgehub_lakehouse.gold.dim_date"
)


display(spark.table("knowledgehub_lakehouse.gold.dim_date").limit(10))


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

dept_src = spark.table("knowledgehub_lakehouse.reference.departments")

dept_clean = (
    dept_src
    .select(
        F.trim(F.col("department")).alias("department_name"),
        F.col("owner_group").cast("string"),
        F.col("sensitivity_level").cast("string")
    )
)

window_spec = Window.orderBy("department_name")

dim_department = (
    dept_clean
    .withColumn("department_key", F.row_number().over(window_spec))
)

dim_department.write.mode("overwrite").format("delta").saveAsTable("knowledgehub_lakehouse.gold.dim_department")

display(spark.table("knowledgehub_lakehouse.gold.dim_department"))


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

doc_type_src = spark.table("knowledgehub_lakehouse.reference.doc_type_rules")

doc_type_clean = (
    doc_type_src
    .select(
        F.upper(F.trim(F.col("doc_type"))).alias("doc_type"),
        F.col("mandatory_fields").cast("string"),
        F.col("retention_days").cast("int"),
        F.col("allowed_confidentiality").cast("string")
    )
)

window_spec = Window.orderBy("doc_type")

dim_doc_type = (
    doc_type_clean
    .withColumn("doc_type_key", F.row_number().over(window_spec))
)

dim_doc_type.write.mode("overwrite").format("delta").saveAsTable("knowledgehub_lakehouse.gold.dim_doc_type")

display(spark.table("knowledgehub_lakehouse.gold.dim_doc_type"))


In [0]:
from pyspark.sql import functions as F

# Source tables
docs = spark.table("knowledgehub_lakehouse.silver.docs_clean")
dim_department = spark.table("knowledgehub_lakehouse.gold.dim_department")
dim_doc_type = spark.table("knowledgehub_lakehouse.gold.dim_doc_type")

fact_doc_versions = (
    docs
    # -------------------------
    # Join dimensions
    # -------------------------
    .join(
        dim_department,
        docs.department == dim_department.department_name,
        "left"
    )
    .join(
        dim_doc_type,
        docs.doc_type == dim_doc_type.doc_type,
        "left"
    )

    # -------------------------
    # Date surrogate key
    # -------------------------
    .withColumn(
        "date_key",
        F.date_format(F.to_date("updated_ts"), "yyyyMMdd").cast("int")
    )
    .withColumn( "freshness_days", 
                F.datediff(F.current_date(), F.to_date("updated_ts")) ) 
    .withColumn( "missing_metadata_count", 
                ( F.when(F.col("department").isNull(), 1).otherwise(0) + 
                 F.when(F.col("dim_doc_type.doc_type").isNull(), 1).otherwise(0) + 
                 F.when(F.col("confidentiality").isNull(), 1).otherwise(0) ) ) 
    .withColumn( "text_length_bucket", 
                F.when(F.length("doc_text") < 500, "SMALL") .
                when(F.length("doc_text") < 2000, "MEDIUM") .otherwise("LARGE") )

    # -------------------------
    # Final fact projection
    # -------------------------
    .select(
        # Natural document keys
        "doc_id",
        "version_norm",
        "current_version_flag",

        # Surrogate dimension keys
        "department_key",
        "doc_type_key",
        "date_key",

        # metadata
        "freshness_days",
        "missing_metadata_count",
        "text_length_bucket",
    
        # Business attributes
        "doc_title",
        "owner_user",
        "status",
        "confidentiality",
        "policy_region",

        # Audit / lineage
        "source_system",
        "source_file",
        "input_batch",
        "ingest_ts",

        # Timestamps
        "created_ts",
        "updated_ts"
    )
)

display(fact_doc_versions)


In [0]:
fact_doc_versions.write.mode("append").format("delta").option("mergeSchema", "true").saveAsTable("knowledgehub_lakehouse.gold.fact_doc_versions")


In [0]:
# from pyspark.sql import functions as F
# from pyspark.sql.window import Window

# docs = spark.table("knowledgehub_lakehouse.silver.docs_clean")
# dim_department = spark.table("knowledgehub_lakehouse.gold.dim_department")
# dim_doc_type = spark.table("knowledgehub_lakehouse.gold.dim_doc_type")

# window_spec = Window.partitionBy("doc_id", "version_norm") \
#                     .orderBy(F.col("updated_ts").desc(), F.col("ingest_ts").desc())

# src = (
#     docs
#     .withColumn("rn", F.row_number().over(window_spec))
#     .filter("rn = 1")
#     .join(dim_department, docs.department == dim_department.department_name, "left")
#     .join(dim_doc_type, docs.doc_type == dim_doc_type.doc_type, "left")
#     .withColumn("freshness_days", F.datediff(F.current_date(), F.to_date("updated_ts")))
#     .withColumn(
#         "missing_metadata_count",
#         (docs.department.isNull().cast("int") +
#          docs.doc_type.isNull().cast("int") +
#          docs.confidentiality.isNull().cast("int"))
#     )
#     .withColumn(
#         "text_length_bucket",
#         F.when(F.length("doc_text") < 500, "SMALL")
#          .when(F.length("doc_text") < 2000, "MEDIUM")
#          .otherwise("LARGE")
#     )
#     .withColumn(
#         "date_key",
#         F.date_format(F.to_date("updated_ts"), "yyyyMMdd").cast("int")
#     )
#     .select(
#         "doc_id",
#         "version_norm",
#         "current_version_flag",
#         "department_key",
#         "doc_type_key",
#         "date_key",
#         "doc_title",
#         "owner_user",
#         "status",
#         "confidentiality",
#         "policy_region",
#         "source_system",
#         "source_file",
#         "input_batch",
#         "ingest_ts",
#         "created_ts",
#         "updated_ts",
#         "freshness_days", 
#         "missing_metadata_count",
#         "text_length_bucket"
#     )
# )

# src.createOrReplaceTempView("src_fact_docs")

# spark.sql("""
# MERGE INTO knowledgehub_lakehouse.gold.fact_doc_versions tgt
# USING src_fact_docs src
# ON tgt.doc_id = src.doc_id
# AND tgt.version_norm = src.version_norm

# WHEN MATCHED AND src.updated_ts > tgt.updated_ts THEN
#   UPDATE SET *

# WHEN NOT MATCHED THEN
#   INSERT *
# """)

In [0]:
%sql
SELECT doc_id, COUNT(*) 
FROM knowledgehub_lakehouse.gold.fact_doc_versions
GROUP BY doc_id
ORDER BY COUNT(*) DESC;


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# -------------------------------------------------
# 1. Read source users (latest snapshot)
# -------------------------------------------------
src_users = (
    spark.table("knowledgehub_lakehouse.reference.users")
    .select(
        F.col("user_id").cast("string"),
        F.col("role").cast("string"),
        F.col("department").cast("string"),
        F.col("region").cast("string")
    )
    .dropDuplicates(["user_id"])
)

# -------------------------------------------------
# 2. Read existing dimension (if exists)
# -------------------------------------------------
if spark.catalog.tableExists("knowledgehub_lakehouse.gold.dim_user"):
    tgt_users = spark.table("knowledgehub_lakehouse.gold.dim_user")
else:
    tgt_users = None

# -------------------------------------------------
# 3. If table does not exist → Initial load
# -------------------------------------------------
if tgt_users is None:
    dim_user = (
        src_users
        .withColumn("user_key", F.monotonically_increasing_id())
        .withColumn("effective_from", F.current_date())
        .withColumn("effective_to", F.lit(None).cast("date"))
        .withColumn("is_current", F.lit(True))
    )

    dim_user.write.mode("overwrite").format("delta").saveAsTable(
        "knowledgehub_lakehouse.gold.dim_user"
    )

else:
    # -------------------------------------------------
    # 4. Identify changed records (SCD Type 2)
    # -------------------------------------------------
    current_dim = tgt_users.filter(F.col("is_current") == True)

    changes = (
        src_users.alias("src")
        .join(
            current_dim.alias("tgt"),
            on="user_id",
            how="inner"
        )
        .filter(
            (F.col("src.role") != F.col("tgt.role")) |
            (F.col("src.department") != F.col("tgt.department")) |
            (F.col("src.region") != F.col("tgt.region"))
        )
        .select(F.col("tgt.user_key"), F.lit(True).alias("changed_flag"))
    )

    # -------------------------------------------------
    # 5. Expire old records
    # -------------------------------------------------
    expired = (
        tgt_users
        .join(changes, on="user_key", how="left")
        .withColumn(
            "effective_to",
            F.when(F.col("changed_flag") == True, F.current_date() - F.expr("INTERVAL 1 DAY"))
             .otherwise(F.col("effective_to"))
        )
        .withColumn(
            "is_current",
            F.when(F.col("changed_flag") == True, F.lit(False))
             .otherwise(F.col("is_current"))
        )
        .drop("changed_flag")
    )

    # -------------------------------------------------
    # 6. Insert new versions for changed & new users
    # -------------------------------------------------
    new_versions = (
        src_users.alias("src")
        .join(
            current_dim.alias("tgt"),
            on="user_id",
            how="left"
        )
        .filter(
            F.col("tgt.user_id").isNull() |  # brand new user
            (F.col("src.role") != F.col("tgt.role")) |
            (F.col("src.department") != F.col("tgt.department")) |
            (F.col("src.region") != F.col("tgt.region"))
        )
        .select(
            F.monotonically_increasing_id().alias("user_key"),
            F.col("src.user_id"),
            F.col("src.role"),
            F.col("src.department"),
            F.col("src.region"),
            F.current_date().alias("effective_from"),
            F.lit(None).cast("date").alias("effective_to"),
            F.lit(True).alias("is_current")
        )
    )

    # -------------------------------------------------
    # 7. Union expired + new versions and overwrite
    # -------------------------------------------------
    final_dim_user = expired.unionByName(new_versions)
    
    # Break lineage to avoid reading from table being overwritten
    final_dim_user.createOrReplaceTempView("temp_dim_user")
    
    spark.table("temp_dim_user").write.mode("overwrite").format("delta").saveAsTable(
        "knowledgehub_lakehouse.gold.dim_user"
    )

# -------------------------------------------------
# 8. Validate
# -------------------------------------------------
display(
    spark.table("knowledgehub_lakehouse.gold.dim_user")
    .orderBy("user_id", "effective_from")
)

In [0]:
from pyspark.sql import functions as F

# --------------------------------------------------
# Read tables
# --------------------------------------------------
events = spark.table("knowledgehub_lakehouse.silver.access_events_clean")

dim_user = (
    spark.table("knowledgehub_lakehouse.gold.dim_user")
    .filter("is_current = true")
    .select("user_id", "user_key")
)

# --------------------------------------------------
# Filter non-orphan & deduplicate
# --------------------------------------------------
events_dedup = (
    events
    .filter("orphan_doc_flag = 0")
    .dropDuplicates(["event_id"])
)

# --------------------------------------------------
# Join with dim_user & add date_key
# --------------------------------------------------
fact_access_events = (
    events_dedup
    .join(dim_user, on="user_id", how="left")
    .withColumn("date_key", F.date_format(F.to_date("event_ts"), "yyyyMMdd").cast("int"))
    .select(
        "doc_id",
        "action",
        "client_type",
        "event_id",
        "event_ts",
        "result_count",
        "search_query",
        "user_key",                # surrogate key
        "_rescued_data",
        "ingest_ts",
        "source_file",
        "input_batch",
        "orphan_doc_flag",
        "search_success_flag",
        "date_key"
    )
)

# --------------------------------------------------
# Write to GOLD fact table
# --------------------------------------------------
fact_access_events.write.mode("append").format("delta").option("mergeSchema", "true").saveAsTable(
    "knowledgehub_lakehouse.gold.fact_access_events"
)

display(fact_access_events)


In [0]:
# %sql
# MERGE INTO knowledgehub_lakehouse.gold.fact_access_events tgt
# USING (
#   SELECT 
#     src.doc_id,
#     src.action,
#     src.client_type,
#     src.event_id,
#     src.event_ts,
#     src.result_count,
#     src.search_query,
#     u.user_key,
#     src._rescued_data,
#     src.ingest_ts,
#     src.source_file,
#     src.input_batch,
#     src.orphan_doc_flag,
#     src.search_success_flag,
#     CAST(date_format(to_date(src.event_ts), 'yyyyMMdd') AS INT) AS date_key
#   FROM knowledgehub_lakehouse.silver.access_events_clean src
#   LEFT JOIN knowledgehub_lakehouse.gold.dim_user u
#     ON src.user_id = u.user_id AND u.is_current = true
# ) src
# ON tgt.event_id = src.event_id
# WHEN NOT MATCHED AND src.orphan_doc_flag = 0 THEN
#   INSERT (
#     doc_id, action, client_type, event_id, event_ts, result_count, search_query, user_key, _rescued_data, ingest_ts, source_file, input_batch, orphan_doc_flag, search_success_flag, date_key
#   )
#   VALUES (
#     src.doc_id, src.action, src.client_type, src.event_id, src.event_ts, src.result_count, src.search_query, src.user_key, src._rescued_data, src.ingest_ts, src.source_file, src.input_batch, src.orphan_doc_flag, src.search_success_flag, src.date_key
#   );


In [0]:
from pyspark.sql import functions as F

usage_daily = (
    spark.table("knowledgehub_lakehouse.gold.fact_access_events")
    .withColumn("event_date", F.to_date("event_ts"))
    .groupBy("doc_id", "event_date")
    .agg(
        F.count("*").alias("total_events"),
        F.countDistinct("user_key").alias("unique_users"),
        F.sum(F.when(F.col("action") == "SEARCH", 1).otherwise(0)).alias("searches"),
        F.sum(F.when(F.col("action") == "VIEW", 1).otherwise(0)).alias("views"),
        F.sum(F.when(F.col("action") == "DOWNLOAD", 1).otherwise(0)).alias("downloads")
    )
)


In [0]:
usage_daily.write.mode("append").format("delta").option("mergeSchema", "true") \
    .partitionBy("event_date") \
    .saveAsTable("knowledgehub_lakehouse.gold.fact_doc_daily_usage")


In [0]:
%sql
OPTIMIZE knowledgehub_lakehouse.gold.fact_doc_versions
ZORDER BY (doc_id);

OPTIMIZE knowledgehub_lakehouse.gold.fact_access_events
ZORDER BY (doc_id, event_ts);

OPTIMIZE knowledgehub_lakehouse.gold.fact_doc_daily_usage
ZORDER BY (doc_id);


In [0]:
%sql
SELECT doc_id, SUM(total_events) AS total_events
FROM knowledgehub_lakehouse.gold.fact_doc_daily_usage
GROUP BY doc_id
ORDER BY total_events DESC
LIMIT 10;

In [0]:
%sql
DESCRIBE HISTORY knowledgehub_lakehouse.gold.fact_doc_versions;


In [0]:
%sql
SELECT
  doc_id,
  version_norm,
  current_version_flag,
  updated_ts
FROM knowledgehub_lakehouse.gold.fact_doc_versions
VERSION AS OF 1
WHERE doc_id = 'DOC00875';
